In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import warnings
warnings.simplefilter('ignore')

In [3]:
#sub_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
filename = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/2class/2_df_sub14.csv'
df = pd.read_csv(filename)
df = df.dropna(axis=0)
df

,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,1.499222,-0.011673,2.809906,32.851776,2.146912,0.0
1,1.499222,-0.015060,2.825165,32.929749,1.878357,0.0
2,1.499084,-0.002609,2.797699,32.845764,2.174377,0.0
3,1.498947,-0.002106,2.817535,32.862244,2.165222,0.0
4,1.499130,-0.009247,2.827072,32.866760,2.195740,0.0
...,...,...,...,...,...,...
1558896,0.025314,-0.021378,3.066254,34.142822,-3.422546,0.0
1558897,0.012955,-0.022110,3.058243,34.151978,-3.404236,0.0
1558898,0.005447,-0.002197,3.060150,34.100128,-3.422546,0.0
1558899,0.001373,-0.017487,3.062820,34.124512,-3.440857,0.0


In [4]:
X = df.drop('w_label', axis = 1)
y = df.w_label

In [5]:
X

,c_ecg,c_emg,c_eda,c_temp,c_resp
0,1.499222,-0.011673,2.809906,32.851776,2.146912
1,1.499222,-0.015060,2.825165,32.929749,1.878357
2,1.499084,-0.002609,2.797699,32.845764,2.174377
3,1.498947,-0.002106,2.817535,32.862244,2.165222
4,1.499130,-0.009247,2.827072,32.866760,2.195740
...,...,...,...,...,...
1558896,0.025314,-0.021378,3.066254,34.142822,-3.422546
1558897,0.012955,-0.022110,3.058243,34.151978,-3.404236
1558898,0.005447,-0.002197,3.060150,34.100128,-3.422546
1558899,0.001373,-0.017487,3.062820,34.124512,-3.440857


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

False


In [9]:
import sklearn as sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
pd.Series(y).value_counts()

0.0    1086401
1.0     472500
Name: w_label, dtype: int64

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# **Random Forest**

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators = 50, min_samples_leaf = 5)

In [15]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train, y_train, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([ 96.54522872, 107.52766013,  86.64575076,  90.87465811,
         93.90581775,  88.22456574,  86.22079539,  84.17959905,
         97.21152687,  87.73872304]),
 'score_time': array([0.77049732, 0.76068234, 0.73750401, 0.77854013, 0.76798081,
        0.75346708, 0.75745153, 0.74734354, 0.75662374, 0.77240252]),
 'test_accuracy': array([0.97477922, 0.97514273, 0.97521757, 0.97488614, 0.9743195 ,
        0.97428742, 0.97496098, 0.9748113 , 0.97405222, 0.97447987]),
 'test_f1': array([0.95854203, 0.9591697 , 0.95933333, 0.95873083, 0.95781228,
        0.9577633 , 0.95886031, 0.95862311, 0.95745166, 0.958075  ]),
 'train_accuracy': array([0.9871348 , 0.98708729, 0.98717163, 0.98709798, 0.9870469 ,
        0.987155  , 0.98710035, 0.98710867, 0.98727617, 0.98716094]),
 'train_f1': array([0.97884328, 0.97876779, 0.97890017, 0.97878781, 0.97870346,
        0.97888049, 0.97878633, 0.97879972, 0.9790753 , 0.97888749])}

# **Random Forest(SMOTE-ENN)**

In [16]:
from imblearn.combine import SMOTEENN

SMOTEENN = SMOTEENN(random_state=0)
x_train_over,y_train_over = SMOTEENN.fit_resample(x_train,y_train)

print('SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTEENN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트:  (935340, 5) (935340,)
SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트:  (1238558, 5) (1238558,)
SMOTEENN 적용 후 레이블 값 분포: 
 1.0    625219
0.0    613339
Name: w_label, dtype: int64


In [17]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train_over, y_train_over, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([140.52872133, 146.92649603, 141.08940029, 142.89603782,
        158.79942226, 144.95235658, 160.0446496 , 160.52597618,
        163.67191982, 140.60025096]),
 'score_time': array([0.78006577, 0.76260304, 0.77518511, 0.76861668, 0.76334143,
        0.7973609 , 0.7879796 , 0.7783463 , 0.76634812, 0.76006079]),
 'test_accuracy': array([0.99346822, 0.99365392, 0.99325023, 0.99308875, 0.99315334,
        0.9932583 , 0.9930726 , 0.99377503, 0.99351661, 0.99319365]),
 'test_f1': array([0.99354118, 0.99372515, 0.99333057, 0.99317047, 0.99323149,
        0.99333711, 0.99315189, 0.99384564, 0.9935944 , 0.99327134]),
 'train_accuracy': array([0.99621782, 0.99617297, 0.99614964, 0.99613798, 0.99618373,
        0.99612363, 0.99612094, 0.99614067, 0.99612094, 0.99615144]),
 'train_f1': array([0.99625909, 0.99621502, 0.99619194, 0.99618045, 0.99622564,
        0.99616606, 0.99616356, 0.99618313, 0.99616369, 0.99619363])}